In [1]:
# from: https://github.com/michhar/python-jupyter-notebooks/blob/master/cognitive_services/Computer_Vision_API.ipynb

In [2]:
import time 
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import json
# from urlparse import urlparse
# from urllib import urlencode
#'from urllib2 import Request, urlopen
from urllib.parse import urlparse
from urllib.parse import urlencode
import urllib.request as Request
from urllib.request import urlopen
#from httplib import HTTPSConnection 
from http.client import HTTPSConnection

# Import library to display results
import matplotlib.pyplot as plt
%matplotlib inline 
# Display images within Jupyter

In [11]:
def processRequest( url, json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:
        # Use the requests library to make the POST call
        response = requests.request( 'post', 
                                    url, 
                                    json=json, 
                                    data=data, 
                                    headers=headers, 
                                    params=params )

        if response.status_code == 429: 
            print( "Message: %s" % ( response.json()['error']['message'] ) )
            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break
                
        # We have a successful response, but let's do some tests on response data
        elif response.status_code == 200 or response.status_code == 201:
            if 'content-length' in response.headers and \
                int(response.headers['content-length']) == 0: 
                    
                result = None
                
            elif 'content-type' in response.headers and \
                isinstance(response.headers['content-type'], str):
                    
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None
                    
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json() ) )

        break
        
    return result

In [12]:
# Paste in your API key here to the 'subscription_key' variable or create a config.json

# f = urlopen('https://gist.githubusercontent.com/michhar/03b38c07be7a3a26aa83005e4b38d4f6/raw/6a214f22f63f2e28905f48c991438736bcc77eef/config.json')

# CONFIG=json.loads(f.read())
# CONFIG = {"subscription_key_computer_vision": "66c69ad2f7624d1c82caf7eb3eb3dd11"} # example
CONFIG = {"subscription_key_computer_vision": "7e183a9b776743d39a7560b9c67ab633"}  # Training Key
# CONFIG = {"subscription_key_computer_vision": "9854d7f3b8974f11999e2fb3059d8c71"}  # Prediction Key
# grab key from json in config
subscription_key = CONFIG['subscription_key_computer_vision']

print("Subscription key =", subscription_key)

Subscription key = 7e183a9b776743d39a7560b9c67ab633


In [13]:
""" example

# Variables

# Make sure to take your Endpoint from Overview page of API in Azure Portal
_url = 'https://westeurope.api.cognitive.microsoft.com/vision/v1.0/describe' # change the ending to 'analyze' or 'describe'
_key = subscription_key

_maxNumRetries = 10
*/
"""

CustomVisionURL = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v1.1/Prediction/5f1bedce-e43b-49b8-bb7b-a2fea704fb0b/image?iterationId=f48f00c4-f4da-4e7f-8337-b18bd8536b98'
PredictionKey = '9854d7f3b8974f11999e2fb3059d8c71'


In [36]:
# ! curl -O https://challenge.blob.core.windows.net/challengefiles/gear_images.zip
print("Images file downloaded")

# ! unzip gear_images.zip
! ls -R gear_images
print("Files unzipped")


Images file downloaded
gear_images:
axes   carabiners  gloves	      harnesses  insulated_jackets  rope
boots  crampons    hardshell_jackets  helmets	 pulleys	    tents

gear_images/axes:
100172.jpeg		  2022015_701_main.jpg
10036053x1012980_zm.jpeg  2022022_004_main.jpg
10045714_zm.jpeg	  2022035_601_main.jpg
100937.jpeg		  2027570_007_main.jpg
100938.jpeg		  2027573_007_main.jpg
10101173x1012980_zm.jpeg  2034782_001_main.jpg
10188438x1011898_zm.jpeg  2136EfcKEsL._AC_US436_QL65_ (2).jpg
10189964_zm.jpeg	  2136EfcKEsL._AC_US436_QL65_.jpg
10189970_zm.jpeg	  21HnHt+LMDL._AC_US436_QL65_ (2).jpg
10189971_zm.jpeg	  21HnHt+LMDL._AC_US436_QL65_.jpg
10215531x1028317_zm.jpeg  3121G7pNU1L._AC_US436_QL65_.jpg
10258915x1152209_zm.jpeg  316Egv7uPML._AC_US436_QL65_.jpg
10260981_zm.jpeg	  316lIqSK-tL._AC_US436_QL65_.jpg
10260983_zm.jpeg	  31AjDH9RQyL._AC_US436_QL65_.jpg
10285571x1013345_zm.jpeg  31ErQ34PXbL._AC_US436_QL65_.jpg
10285573x1013345_zm.jpeg  31FQHojVcNL._AC_US436_QL65_.jpg
10291030_zm.jpeg	 

Files unzipped


In [49]:
'''from PIL import Image # , ImageFilter
#Read image
im = Image.open( 'gear_images/hardshell_jackets/10116634x1038116_zm.jpeg')
im
'''

PredictionURL = 'https://southcentralus.api.cognitive.microsoft.com/customvision/v1.1/Prediction/5f1bedce-e43b-49b8-bb7b-a2fea704fb0b/url?iterationId=f48f00c4-f4da-4e7f-8337-b18bd8536b98'
PredictionKey = '9854d7f3b8974f11999e2fb3059d8c71'
# raNDOM IMAGE FROM THE 'NET
imageURL = 'http://demandware.edgesuite.net/sits_pod41/dw/image/v2/ABBM_PRD/on/demandware.static/-/Sites-patagonia-master/default/dw8c3bd24c/images/hi-res/84212_BLK.jpg?sw=750&sh=750&sm=fit&sfrm=png'
from IPython.display import display, Image
img = Image(url=imageURL)
display(img)
# and in JSON:
img = { 'url': imageURL }
headers = {"Prediction-Key": PredictionKey, 'Content-Type': 'application/json'}
params = {}

#result = processRequest(CustomVisionURL, img, '', headers, params)
result = requests.request('post', PredictionURL, json=img, headers=headers)
# print(type(result))
# print(type(result.content))
result = str(result.content)[2:-1]
# print(result)
output = json.loads(result)

predictions = output["Predictions"]
predictions
for pred in predictions:
    print("Prediction:", pred["Tag"], "probability:", pred["Probability"])

Prediction: insulated jacket probability: 0.999973834
Prediction: hardshell jacket probability: 4.547137e-06
